In [29]:
import os, config, pickle, numpy as np, pandas as pd

# Extracting lxmx data for analysis

You will need the following packages installed to be able to run the notebook.

In [30]:
LXMX = pd.read_excel(os.path.join(config.DATA_DIR, config.LXMX_FILENAME), index_col=None, header=None)

## Find the species: 
* store their names 
* slice indices in the DataFrame

In [31]:
def find_species(df):
    species_ix = np.where(pd.isna(df[config.SPECIES_COL]) == False)[0]
    prev_index = -100
    fixed_species_ix = []
    for ix in species_ix:
            if ix-1 == prev_index:
                continue
            else:
                fixed_species_ix.append(ix)
                prev_index = ix
    species_names = list(df[config.SPECIES_COL][fixed_species_ix])
    fixed_species_ix.append(len(df))
    species_ix = np.array(fixed_species_ix)
    species = {}
    for i in range(len(species_names)):
        s = species_names[i]
        species[s] = (species_ix[i], species_ix[i+1]) 
    return species

In [32]:
SPECIES_ix = find_species(LXMX)

## Auxillary classes

#### Data
The Data object stores the matrices, the life history variables, and the auhtor/collector of the data entry as mentioned in the xls.

In [43]:
class Data:
    def __init__(self, df):
        cols = ['age', 'l(a)', 'f(a)', 'p(a)']
        L = len(df)
        df.index = range(L)
        self.matrix =df.loc[0:L-2, 0:3]
        self.matrix.columns = cols
        self.matrix.index = self.matrix['age']
        self.matrix = self.matrix[cols[1:]]
        self.matrix = self.matrix.fillna(0.)
        self.author = df[L-1:L][7].item()
        self.vars = df.loc[0:1, 5:6]
        self.vars.columns = ['0', '1']
       
    @property
    def ages(self):
        return self.matrix.index.to_numpy()
        
    def __repr__(self):
        return self.author

#### Species
Species object contains the raw DataFrame slice of all of the species data entries, as well as the name and notes (if present).

In [44]:
class Species:
    def __init__(self, name, df, lookup):
        self.name = name
        self.df = df[lookup[name][0]:lookup[name][1]]
        self.df.index = range(len(self.df))
        self.notes = df[lookup[name][0]+1:lookup[name][0]+2][4].item()
        if pd.isna(self.notes):
            self.notes = None
        Species.clean_data(self, lookup[name][1])
        
    def __repr__(self):
        return '{}: {} data entries, notes: {}'.format(self.name, len(self.data), self.notes)
    
    @staticmethod
    def clean_data(s, end):
        ix = list(np.where(pd.isna(s.df[6]) == False)[0][::2])
        N = len(ix)
        ix.append(end)
        s.data=[]
        for n in range(N):
            s.data.append(Data(s.df[ix[n]:ix[n+1]]))

In [45]:
SPECIES = {}
for s in SPECIES_ix:
    SPECIES[s] = Species(s, LXMX, SPECIES_ix)

In [49]:
save = True
if save:
    with open(os.path.join(config.OUTPUT_DIR, 'species.pkl'), 'wb') as f:
        pickle.dump(SPECIES, f)

## Working with the Species objects

You can list the keys with the following code (splice at the end is for the sake of limiting output), 

In [47]:
list(SPECIES.keys())[0:10]

['Spermophilus columbianus',
 'Spermophilus lateralis',
 'Theropithecus gelada',
 'Yellow-bellied marmot',
 'Spermophilus armatus',
 'Tursiops truncatus',
 'Papio cynocephalus',
 'Helogale parvula',
 'Callorhinus ursinus',
 'Phacochoerus aethiopicus']

and access the Species object as such.

In [48]:
SPECIES['Red deer'].data[0].matrix

,l(a),f(a),p(a)
age,,,
1.0,0.615000,0.000,0.902
2.0,0.554730,0.000,0.962
3.0,0.533650,0.086,0.956
4.0,0.510170,0.202,0.956
5.0,0.487720,0.241,0.949
6.0,0.462850,0.209,0.939
7.0,0.434610,0.212,0.971
8.0,0.422010,0.215,0.982
9.0,0.414410,0.219,0.942


In [41]:
SPECIES['Capreolus capreolus'].data

[Kaluzinski 1982, Johnson 1982]

In [25]:
SPECIES['Capreolus capreolus'].data[0].matrix

,l(a),f(a),p(a)
age,,,
0.0,1.000,0.000,0.51
1.0,0.502,0.000,0.95
2.0,0.476,1.000,0.93
3.0,0.442,0.885,0.93
4.0,0.395,1.000,0.88
5.0,0.349,1.270,0.87
6.0,0.297,1.000,0.83
7.0,0.246,1.000,0.8
8.0,0.170,0.890,0.56


In [26]:
SPECIES['Capreolus capreolus'].data[0].matrix.to_numpy()

array([[1.0, 0.0, 0.51],
       [0.502, 0.0, 0.95],
       [0.476, 1.0, 0.93],
       [0.442, 0.885, 0.93],
       [0.395, 1.0, 0.88],
       [0.349, 1.27, 0.87],
       [0.297, 1.0, 0.83],
       [0.246, 1.0, 0.8],
       [0.17, 0.89, 0.56],
       [0.063, 0.865, 0.371],
       [0.0, 0.8, 0]], dtype=object)

In [27]:
SPECIES['Capreolus capreolus'].data[0].ages

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., 10., 15.])